## *Learning from Data - Workshop Session 3*
# <br> <center> Data Preprocessing, Regression & Bias-variance Analysis</center>

**Intended Learning Outcomes (ILOs)** for Workshop Session 3:
Upon completion of this workshop session, you will be able to:
+ Implement the data preprocessing actions (including data cleaning, data transformation, and data virtualisation) on real-world dataset, per the requirement of target data analytics task
+ Develop the linear and polynomial regression models based on real-world dataset
+ Apply the techniques of L1 regularisation and cross validation with a right bias-variance balance

**Dataset** for Advanced Analysis: ["IMDB-top Netflix Programme Dataset"](https://www.kaggle.com/datasets/bharatnatrayn/movies-dataset-for-feature-extracion-prediction/ "IMDB-top Netflix Movies Dataset") (click to download the dataset from Kaggle)
<br> <br> The dataset is scrapped from IMDB about Netflix movies and tv shows.This dataset needs clever programming knowledge for feature extraction also building a Netflix programme recommendation system either other prediction models.
<br> <br> **About the Dateset:**
+ MOVIES: The name of movie or tv show
+ YEAR: The year of movie or tv shows telecast for audience
+ GENRE: The type of movie or tv shows
+ RATING: The audience thought about movie or tv show given
+ ONE-LINE: The short description about movie or tv show for audiences first impression
+ STARS: The casting of art which is made talks between audience 
+ VOTES: The number of audience expressing their view
+ RUNTIME: The duration for runing time of art
+ GROSS: The total amount of money earned in worldwide

**Note:**
+ Please run the code below in your Python programming environment. Running on a Jupyter notebook is **highly recommended**, with every few lines of code in a different cell within the notebook.
+ Please select the practical tasks according to your leaning capacity. Feel free to try on the **optional** / **bonus** task, upon the completion of other practices.

### Practical Task A: Review the Structure of Movie Dataset
**1.**	Load the file `movies_dataset.csv` using the python `pandas` library:

**2.** Review the dataset structure

In [1]:
# get shape of data that is (rows count,column count)


In [2]:
# get 1st few rows from df


In [3]:
# get the mata information of df


In [4]:
# count of null values column wise


**3.** **Data Cleaning** - Drop the duplicated data pieces

In [5]:
# drop duplicated rows


In [6]:
# drop duplicated rows with the same movie/tv show name


**4. Data Cleaning** - Remove rows where YEAR, GENRE, RATING, VOTES, RUNTIME AND STARS is NaN

In [7]:
# get the NaN data items


In [8]:
# remove the NaN data items


**5. Data Transformation** - Transform the features in columns that include '\n'<br> &nbsp; &nbsp; <u>**Hint:**</u> Python String replace() Method

**6. Data Transformation** - Remove commas and convert the VOTES column to numeric

**7. Data visulisation** - Analyse the GENRE column

In [9]:
# count unique genres


In [10]:
# create a bar chart


In [11]:
# create a pie chart


In [12]:
# create a pie chart for top-10 genres


**8. Data Visualisation** - Analyse the correlation amongst the columns of RATING, RUNTIME, and VOTES

In [13]:
# get the correlation matrix of these three columns


In [14]:
# create a heatmap


**9. Data Transformation [ Optional ]** - Transform the STARS column into separate columns (i.e., DIRECTOR, STARS)

In [15]:
# review the STARS column


<u>**Hint:**</u> re - Regular expression operations &nbsp; https://docs.python.org/3/library/re.html <br>

In [15]:
#creating new columns


In [ ]:
# review the dataset after transformation


### Practical Task B: Analyse the Feature Correlation between RATING and GROSS

**1.** **Data Proprocessing** per the requirement of practical task B

In [16]:
# count of NaN values column wise


In [17]:
# remove rows where GROSS is NaN


In [18]:
# transform the GROSS from string to float


**2.** Make a scatterplot for the RATING and the GROSS columns

**3.** Conduct linear regression for the RATING and the GROSS columns

**4.** Conduct <u>polynomial regression (degree = 5)</u> for the RATING and the GROSS columns

In [19]:
# Fitting Polynomial Regression to the dataset


In [20]:
# Visualising the Polynomial Regression results


**5.** Conduct <u>polynomial regression (degree = 20)</u> for the RATING and the GROSS columns

In [21]:
# Fitting Polynomial Regression to the dataset


In [22]:
# Visualising the Polynomial Regression results


**6.** Conduct <u>ridge regression (degree = 20, alpha=1000000)</u> for the RATING and the GROSS columns

In [23]:
# Fitting Ridge Regression to the dataset


In [24]:
# Visualise the results


## Bonus Task [Challenging]

**Reflection:** In the Practical Task B, the R2 score achieved by ridge regression is still relatively low. Can we improve the gross prediction performance by incorporating more correlation factors (such as, YEAR, GENRE, DIRECTOR, STARS)? Feel free to have a try!

**Inspiration:** <br>
+ **Q1:** How to transform the data representation of GENRE, DIRECTOR, STARS (in string form)?
+ **Q2:** How to build the regression model from multiple feature columns to target prediction column (i.e., GROSS)?
+ **Q3:** How to visualise the regression model from multiple feature columns to target prediction column (i.e., GROSS)?